# Phi-Engine: Calibration Demo

Demonstrates precision instrument calibration using D-space linearisation.

**Pipeline**: Raw readings -> D-space transform -> Sum-rule validation -> Correction

In [ ]:
import sys
sys.path.insert(0, '../..')

from phi_engine import PhiEngine
from phi_engine.adapters.calibration import CalibrationAdapter
from phi_engine.adapters.sensor_fusion import SensorFusionAdapter

engine = PhiEngine()
engine.register_adapter('calibration', CalibrationAdapter())
engine.register_adapter('sensor_fusion', SensorFusionAdapter())

## 1. Single Instrument Calibration

Simulate a thermometer with systematic positive drift.

In [ ]:
# Simulated readings with +2% systematic drift
reference = 100.0  # known reference temperature
readings = [101.8, 102.1, 101.9, 102.3, 102.0, 101.7, 102.2, 101.95, 102.05, 101.85]

result = engine.run('calibration', {
    'readings': readings,
    'reference': reference,
    'instrument': 'thermometer-A1',
})

print(f"Consistency Score: {result['consistency_score']:.6f}")
print(f"Drift: {result['drift']}")
print("Recommendations:")
for r in result['recommendations']:
    print(f"  - {r}")

## 2. View Corrected Values

In [ ]:
corrected = result.get('corrected_values', [])

print(f"{'Reading':>8s}  {'Raw':>10s}  {'Corrected':>10s}  {'Delta':>10s}")
print("-" * 42)
for i, (raw, corr) in enumerate(zip(readings, corrected)):
    print(f"{i+1:>8d}  {raw:>10.4f}  {corr:>10.4f}  {corr-raw:>+10.6f}")

print(f"\nMean raw:       {sum(readings)/len(readings):.4f}")
print(f"Mean corrected: {sum(corrected)/len(corrected):.4f}")
print(f"Reference:      {reference:.4f}")

## 3. D-Space Residual Visualisation

In [ ]:
from phi_engine.core import D

d_ref = D(reference)
d_readings = [D(r) for r in readings]
d_residuals = [d - d_ref for d in d_readings]

print("D-space residuals (should cluster near 0 for well-calibrated):")
for i, res in enumerate(d_residuals):
    bar = '#' * int(abs(res) * 5000)
    sign = '+' if res > 0 else '-'
    print(f"  [{i+1:2d}] {res:+.8f}  {sign}{bar}")

## 4. Multi-Sensor Fusion

Fuse readings from 3 sensors with different precisions.

In [ ]:
fusion_result = engine.run('sensor_fusion', {
    'sensors': [
        {
            'name': 'high_precision',
            'readings': [100.01, 100.02, 99.99, 100.01, 100.00],
            'weight': 1.0,
        },
        {
            'name': 'medium_precision',
            'readings': [100.1, 99.9, 100.2, 99.8, 100.0],
            'weight': 1.0,
        },
        {
            'name': 'low_precision',
            'readings': [101.0, 99.0, 102.0, 98.5, 100.5],
            'weight': 1.0,
        },
    ],
    'reference': 100.0,
})

print(f"Fused estimate: {fusion_result['fused_estimate']}")
print(f"Consistency:    {fusion_result['consistency_score']:.6f}")
print("\nSensor quality ranking:")
for sq in fusion_result.get('sensor_quality', []):
    print(f"  {sq['sensor']:<20s}  quality={sq['quality']:.4f}  D_std={sq['d_std']:.6f}")

print("\nRecommendations:")
for r in fusion_result.get('recommendations', []):
    print(f"  - {r}")